### IMPORT LIBRARIES

In [1]:
import sqlalchemy
import pandas as pd
import requests
import json
import time
import mysql.connector

In [2]:
from bs4 import BeautifulSoup as bs

### CREATING CONNECTOR

In [3]:
def getconn():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="dogThinh123!",
        database="goodreads"
    )
    return conn

In [4]:
pool = sqlalchemy.create_engine(
    "mysql+mysqlconnector://",
    creator=getconn,
)

with pool.connect() as db_conn:
    results = db_conn.execute(sqlalchemy.text("SELECT NOW()")).fetchone()
    print("Current time: ", results[0])

Current time:  2023-05-08 23:34:33


### EDA

In [5]:
id_list = pd.read_sql_query(sqlalchemy.text("SELECT goodreads_book_id FROM books"), con=pool.connect())
id_list.head()

,goodreads_book_id
0,2767052
1,3
2,41865
3,2657
4,4671


In [6]:
goodreads_book_id_list = id_list['goodreads_book_id'].tolist()
goodreads_book_id_list[:5]

['2767052', '3', '41865', '2657', '4671']

In [7]:
def get_description(id):
  response = requests.get(f'https://www.goodreads.com/book/show/{id}')
  soup = bs(response.content, 'html.parser')
  span = soup.select_one('span[class*="Formatted"]')
  if span is not None:
    content = span.get_text()
    return content
  else:
    return get_description(id)

In [8]:
get_description('2767052')

"Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning?In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen, who lives alone with her mother and younger sister, regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against humanity and life against love."

In [9]:
len(goodreads_book_id_list)

9814

In [10]:
descr_query = sqlalchemy.text(
    "SELECT id FROM book_description;"
)
descr_id = pd.read_sql_query(descr_query,con=pool.connect())
books_df = pd.read_csv(r'D:\project\goodreads\data\goodbooks-10k\books.csv')
id_list = books_df.goodreads_book_id

In [ ]:
id_list_str = [str(id) for id in id_list.tolist()]
id_list_str

In [25]:
set(descr_id.id.tolist()) == set(id_list_str)

True

In [10]:
descriptions_list = pd.DataFrame(columns=['id', 'description'])
for index, id in enumerate(goodreads_book_id_list):
  if index == len(goodreads_book_id_list):
    descriptions_list.to_sql(name="book_description", con=pool, if_exists='append', index=False)
    break
  if index % 500 == 0:
    descriptions_list.to_sql(name="book_description", con=pool, if_exists='append', index=False)
    print(f'\nSAVED AT {index}')
    descriptions_list = pd.DataFrame(columns=['id', 'description'])
  print(f'{index}',end='-')
  description = get_description(id)
  new_record = pd.DataFrame({'id': [id], 'description': [description]})
  descriptions_list = pd.concat([descriptions_list, new_record], ignore_index=True)

Error closing cursor
Traceback (most recent call last):
  File "c:\Users\ACER\.pyenv\pyenv-win\versions\3.10.9\lib\site-packages\sqlalchemy\engine\base.py", line 2186, in _safe_close_cursor
    cursor.close()
  File "c:\Users\ACER\.pyenv\pyenv-win\versions\3.10.9\lib\site-packages\mysql\connector\cursor_cext.py", line 495, in close
    self._cnx.handle_unread_result()
  File "c:\Users\ACER\.pyenv\pyenv-win\versions\3.10.9\lib\site-packages\mysql\connector\connection_cext.py", line 932, in handle_unread_result
    raise InternalError("Unread result found")
mysql.connector.errors.InternalError: Unread result found



SAVED AT 0
0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-

KeyboardInterrupt: 